Database Creation


In [1]:
import requests
from haystack.document_stores import ElasticsearchDocumentStore

#Marcus Aurelios Meditations

data = requests.get('https://raw.githubusercontent.com/jamescalam/transformers/main/data/text/meditations/clean.txt')
text = data.text.split('\n')
text[:3]

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.']

In [2]:
requests.get('http://localhost:9200/_cluster/health').json()

document_store = ElasticsearchDocumentStore(host = 'localhost', username ='elastic', password = 'Ci=SX6m=gTUlH8vDfLfa', index = 'aurelius')

print(requests.get('http://localhost:9200/_cat/indices').text)

yellow open aurelius   tfrAeuIeQeWhWbhjY0GC-g 1 1    0 0  225b  225b
yellow open squad_docs NZuejXoATIa-1XlkmoeaFA 1 1 1204 0 1.2mb 1.2mb
yellow open label      2uS1B1YDRiWC1f6s-OsRsQ 1 1    0 0  225b  225b



In [8]:
data_json = [
    {
        'content': paragraph,
        'meta': {
            'source': 'meditations'
        }
    } for paragraph in text
]

len(data_json)
data_json[:3]

[{'content': 'From my grandfather Verus I learned good morals and the government of my temper.',
  'meta': {'source': 'meditations'}},
 {'content': 'From the reputation and remembrance of my father, modesty and a manly character.',
  'meta': {'source': 'meditations'}},
 {'content': 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
  'meta': {'source': 'meditations'}}]

In [12]:
res = requests.post('http://localhost:9200/aurelius/_delete_by_query',
                   json = {'query':{'match_all':{}}})

In [13]:
res = requests.get('http://localhost:9200/aurelius/_count')
res.json()

{'count': 0,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [14]:
document_store.write_documents(data_json)
print(requests.get('http://localhost:9200/_cat/indices').content)

b'yellow open aurelius   tfrAeuIeQeWhWbhjY0GC-g 1 1  507 2028 1.2mb 1.2mb\nyellow open squad_docs NZuejXoATIa-1XlkmoeaFA 1 1 1204    0 1.2mb 1.2mb\nyellow open label      2uS1B1YDRiWC1f6s-OsRsQ 1 1    0    0  225b  225b\n'


Retriever and Reader Declaration


In [17]:
from haystack.nodes import BM25Retriever
from haystack.nodes import FARMReader

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='aurelius'
)
retriever = BM25Retriever(document_store)

reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=False)


In [19]:
from haystack.pipelines import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [20]:
qa.run(query='What did your grandfather teach?')

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'What did your grandfather teach?',
 'no_ans_gap': -3.1212611198425293,
 'answers': [<Answer {'answer': 'good morals and the government of my temper', 'type': 'extractive', 'score': 0.16657471656799316, 'context': 'From my grandfather Verus I learned good morals and the government of my temper.', 'offsets_in_document': [{'start': 36, 'end': 79}], 'offsets_in_context': [{'start': 36, 'end': 79}], 'document_ids': ['ecc466cb76ed8dedab8752cc84e411b4'], 'meta': {'source': 'meditations'}}>,
  <Answer {'answer': 'thy life under thy mother', 'type': 'extractive', 'score': 0.02412872575223446, 'context': 'Termination of activity, cessation from movement and opinion, and in a sense their death, is no evil. Turn thy thoughts now to the consideration of thy life, thy life as a child, as a youth, thy manhood, thy old age, for in these also every change was a death. Is this anything to fear? Turn thy thoughts now to thy life under thy grandfather, then to thy life under thy mother, then to